# QOSF Task 3 - QSVM

In [1]:
!pip install pennylane 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

np.random.seed(42)

In [26]:
X, y = load_iris(return_X_y=True)

# # pick inputs and labels from the first two classes only,
# # corresponding to the first 100 samples
# X = X[:100]
# y = y[:100]

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)


In [29]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [30]:
y_0 = []
for i in range(0,len(y)):
  if(y[i] != 0):
    y_0.append(1)
  else:
    y_0.append(y[i])

print(y_0)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [31]:
y_1 = []
for i in range(0,len(y)):
  if(y[i] != 1):
    y_1.append(1)
  else:
    y_1.append(0)

print(y_1)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [32]:
y_2 = []
for i in range(0,len(y)):
  if(y[i] != 2):
    y_2.append(1)
  else:
    y_2.append(0)

print(y_2)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled_0 = 2 * (np.array(y_0) - 0.5)
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_scaled, y_scaled_0)

In [35]:
# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled_1 = 2 * (np.array(y_1) - 0.5)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_scaled, y_scaled_1)

In [36]:
# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled_2 = 2 * (np.array(y_2) - 0.5)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_scaled, y_scaled_2)

In [27]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [37]:
print(X_train_2.shape)
print(y_train_2.shape)
print(X_test_2.shape)
print(y_test_2.shape)

(112, 4)
(112,)
(38, 4)
(38,)


In [38]:
print(y_train_2)

[ 1.  1. -1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1. -1.
 -1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1.  1. -1.  1. -1.  1.  1. -1.
  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1.  1.
  1.  1. -1.  1.  1. -1.  1. -1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.
  1.  1. -1. -1.]


In [39]:
n_qubits = len(X_train_2[0])
n_qubits

4

# **One Vs All format**

**For 3 classes**


1.   SVM 1 learns class output = 1 vs class output != 1
2.   SVM 2 learns class output = 2 vs class output != 2
3.   SVM 3 learns class output = 3 vs class output != 3

To predict the output for a new input, just predict with each of SVM and then find the one puts the prediction the farthest into the positive region



In [16]:
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [40]:
print(kernel(X_train_1[0], X_train_1[0]))
print(kernel(X_train_1[0], X_train_1[1]))

1.0
0.008182504838728222


In [41]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [42]:
y_train_svm_0 = SVC(kernel=kernel_matrix).fit(X_train_0, y_train_0)
predictions = y_train_svm_0.predict(X_test_0)
accuracy_score(predictions, y_test_0)

1.0

In [43]:
print(predictions)

[ 1.  1. -1. -1.  1.  1.  1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1.  1.
 -1.  1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1.  1.  1. -1.  1.
 -1. -1.]


In [44]:
y_train_svm_1 = SVC(kernel=kernel_matrix).fit(X_train_1, y_train_1)
predictions = y_train_svm_1.predict(X_test_1)
accuracy_score(predictions, y_test_1)

1.0

In [45]:
y_train_svm_2 = SVC(kernel=kernel_matrix).fit(X_train_2, y_train_2)
predictions = y_train_svm_2.predict(X_test_2)
accuracy_score(predictions, y_test_2)

0.9210526315789473

In [46]:
dev_kernel.num_executions

50408

# Predictions

In [47]:
X, y = load_iris(return_X_y=True)

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

In [48]:
predictions_0 = y_train_svm_0.predict(X_test)
predictions_1 = y_train_svm_1.predict(X_test)
predictions_2 = y_train_svm_2.predict(X_test)

In [49]:
print(predictions_0)
print(predictions_1)
print(predictions_2)

[ 1. -1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.
 -1.  1. -1. -1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1.  1. -1.  1.
 -1.  1.]
[ 1.  1. -1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1. -1. -1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1.
  1.  1.]
[-1.  1.  1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.
  1. -1.  1.  1. -1. -1. -1.  1.  1.  1. -1.  1.  1.  1.  1. -1.  1. -1.
  1. -1.]


In [57]:
print(len(predictions_2))

38


In [50]:
print(y_test)

[2 0 1 1 2 0 1 2 2 0 2 2 1 2 1 1 0 0 0 2 0 0 2 2 2 0 0 0 2 1 1 1 0 2 0 2 0
 2]


In [60]:
y_test_pred = []
for i in range(0,len(predictions_0)):
  if(predictions_0[i] == -1):
    y_test_pred.append(0)
  elif(predictions_1[i] == -1):
    y_test_pred.append(1)
  elif(predictions_2[i] == -1):
    y_test_pred.append(2)
  else:
    y_test_pred.append(2)

print(y_test_pred)

[2, 0, 1, 1, 2, 0, 1, 2, 2, 0, 2, 2, 1, 2, 1, 1, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 1, 1, 1, 0, 2, 0, 2, 0, 2]


In [61]:
print(len(y_test))
print(len(y_test_pred))

38
38


In [62]:
accuracy_score(np.array(y_test_pred), np.array(y_test))

1.0

# Saving the model weights

In [65]:
from joblib import Parallel, delayed
import joblib
  
# Save the model as a pickle in a file
joblib.dump(y_train_svm_0, 'SVM_0.pkl')

['SVM_0.pkl']

In [66]:
joblib.dump(y_train_svm_1, 'SVM_1.pkl')

['SVM_1.pkl']

In [67]:
joblib.dump(y_train_svm_2, 'SVM_2.pkl')

['SVM_2.pkl']